<div class="alert alert-block alert-info">

**TODO:**
* check all texts
* try scaled/unscaled data for trees
* have it without outputs on Github
* read train/test x and y from files
* remove all but one decision tree and one non-decision tree models and make it task to find others and adjust their hyperparameters; provide "solution" script with % load solutions.py (note that this cell has to be run twice then)
* Add info on used data and goal of exercise
* make sure to make it possible to go through in 30 min
    
</div>

# Shallow regression for vector data

Reminder: We are within supervised learning (we have labels/targets that are real values) -> Regression


In this notebook we read the zip code data produced by **01_2_vector_preparations** and create different machine learning models for
predicting the average zip code income from population and spatial features.

We will assesses the models error metrics with a test dataset but also predict the number to all zip codes and write the result to a geopackage for closer inspection.

Notebook contents:
0. Environment preparation
1. Reading the data
2. Function defintion
3. Baseline naive approach
4. Baseline linear regression
5. Gradient Boosting
6. Random Forest
7. Bagging
8. AdaBoost
9. Comparing the models
10. Task 


## 0. Environment preparation

In [1]:
import time
import geopandas as gpd
import pandas as pd
from math import sqrt
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor,ExtraTreesRegressor, AdaBoostRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

import seaborn as sns 

## 1. Reading the data
### 1.1 Define input and output file paths 

In [2]:
preprocessed_data_location = '../preprocessed_data'
dataset_name = 'paavo' # to be changed to vector!

train_dataset_name = os.path.join(preprocessed_data_location,dataset_name,'train_zip_code_data.gpkg')
test_dataset_name = os.path.join(preprocessed_data_location,dataset_name,'test_zip_code_data.gpkg')
train_label_name = os.path.join(preprocessed_data_location,dataset_name,'train_income_labels.pkl')
test_label_name = os.path.join(preprocessed_data_location,dataset_name,'test_income_labels.pkl')


### Relative path to the zip code geopackage file that was prepared by vectorDataPreparations.py
input_geopackage_path = os.path.join(preprocessed_data_location,dataset_name,"zip_code_data_after_preparation.gpkg")

### Output file. You can change the name to identify different regression models
output_geopackage_path = os.path.join(preprocessed_data_location,dataset_name,"median_income_per_zipcode_shallow_model.gpkg")

### 1.2 Reading the data

In [3]:
# read train and test datasets
x_train = gpd.read_file(train_dataset_name)
x_test = gpd.read_file(test_dataset_name)
y_train = pd.read_pickle(train_label_name)
y_test = pd.read_pickle(test_label_name)

,postinumer,nimi,hr_mtu,euref_x,euref_y,pinta_ala,he_vakiy,he_naiset,he_miehet,he_kika,...,Pirkanmaa,Pohjanmaa,Pohjois-Karjala,Pohjois-Pohjanmaa,Pohjois-Savo,Päijät-Häme,Satakunta,Uusimaa,Varsinais-Suomi,geometry
0,00100,Helsinki Keskusta - Etu-Töölö,27577.0,0.467824,0.029171,0.000303,0.679496,0.664961,0.644436,0.288462,...,0,0,0,0,0,0,0,1,0,"MULTIPOLYGON (((385653.893 6671591.048, 385573..."
1,00120,Punavuori,27523.0,0.468701,0.028337,0.000029,0.263362,0.258205,0.249572,0.269231,...,0,0,0,0,0,0,0,1,0,"MULTIPOLYGON (((385316.092 6671076.984, 385279..."
2,00130,Kaartinkaupunki,30479.0,0.469833,0.028373,0.000031,0.055110,0.053156,0.053591,0.288462,...,0,0,0,0,0,0,0,1,0,"MULTIPOLYGON (((386212.111 6671061.262, 386176..."
3,00140,Kaivopuisto - Ullanlinna,29439.0,0.470071,0.027735,0.000102,0.291434,0.295189,0.265798,0.288462,...,0,0,0,0,0,0,0,1,0,"MULTIPOLYGON (((386577.050 6670280.544, 386552..."
4,00150,Eira - Hernesaari,26546.0,0.468366,0.027381,0.000163,0.345510,0.344729,0.320729,0.269231,...,0,0,0,0,0,0,0,1,0,"MULTIPOLYGON (((384846.102 6669565.816, 384823..."


# 2. Function definitions



In [ ]:
# calculating error metrics for regression predictions

def calculate_error_metrics(test_labels,label_predictions, model_name):

    #Asessing the performance of the model with root mean squared error, mean absolute error and coefficient of determination r2
    rmse = sqrt(mean_squared_error(test_labels, label_predictions))
    mae = mean_absolute_error(test_labels, label_predictions)
    r2 = r2_score(test_labels, label_predictions)

    # store them in a dictionary
    metrics_dict = zip(['model','RMSE','MAE','R2'],[model_name,rmse,mae,r2])

    return metrics_dict


def print_and_store_error_metrics(test_labels, label_predictions, model_name, metrics_collection = None):

    metrics_dict = calculate_error_metrics(test_labels, label_predictions)

    # in case these are the first results we want to store in that dataframe, we need to first create it
    if metrics_collection is None:
        metrics_collection = pd.DataFrame(columns=['model','RMSE','MAE','R2'])
 
    metrics_collection = metrics_collection.append(metrics_dict, ignore_index=True)

    print(f"\nError metrics for {model_name} on the test dataset: \n" +
          f"\t Root mean squared error (RMSE): {round(metrics_dict['RMSE'])} \n" +
          f"\t Mean absolute error (MAE): {round(metrics_dict['MAE'])} \n" +
          f"\t Coefficient of determination (R2): {round(metrics_dict['R2'],4)} \n")

    return metrics_collection

# to time the model training we create a function for model training
def train_model(x_train, y_train, model):
    start_time = time.time()  
    print(model)
    model.fit(x_train,y_train)
    print('Model training took: ', round((time.time() - start_time), 2), ' seconds')
    return model

# 3. Baseline naive approach

In order to determine, how well machine learning models perform on our dataset, we create some baseline results.
One way to get baseline results is taking the median of y labels in the training dataset and use this as the prediction for all labels. Very naive, but not a realistic assumption.



In [ ]:

# the median training labels provides the predicted lable
naive_prediction_value = y_train.median()
naive_name = "Naive median prediction"
print(naive_prediction_value)
# the naive prediction value still needs to be repeated to fit with the features
naive_predictions = pd.DataFrame([naive_prediction_value]* y_test.shape[0])
# then we can get some accuracy measures
metrics_collection = print_and_store_error_metrics(y_test,naive_predictions, naive_name)

### 4. Baseline Linear regression 

Another baseline approach for regression is linear regression. Compared to any of the following models it is still easy to compute.

In [30]:

linear = LinearRegression()
linear_name = "Linear Regression"
linear.fit(x_train, y_train)
linear_predictions = linear.predict(x_test)

# get feature importance via their coefficients
feature_names = x_train.columns
model_coefficients = linear.coef_

coefficients_df = pd.DataFrame(data = model_coefficients, index = feature_names, columns = ['Coefficient value'])
print(coefficients_df)
# largest absolute values show the most important features

metrics_collection = print_and_store_error_metrics(y_test,linear_predictions, linear_name, metrics_collection)

                 Coefficient value
euref_x                 245.472829
euref_y               -3138.965227
pinta_ala              -519.491728
he_vakiy            -701947.425354
he_naiset           -704846.512958
...                            ...
Pohjois-Savo            -70.513793
Päijät-Häme            -237.434421
Satakunta              -177.736695
Uusimaa                 -67.035445
Varsinais-Suomi        -252.692260

[116 rows x 1 columns]
0.8501981542418442
0.8904321555840112


## 5. Gradient Boosting Regressor


* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
* https://scikit-learn.org/stable/modules/ensemble.html#regression

In [6]:
grad_boost = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1,verbose=1)
grad_boost_name = "Gradient Boosting Regressor"
grad_boost = train_model(x_train, y_train,grad_boost)
grad_boost_predictions = grad_boost.predict(x_test)
metrics_collection = print_and_store_error_metrics(y_test,grad_boost_predictions, grad_boost_name, metrics_collection)


GradientBoostingRegressor(n_estimators=30, verbose=1)
      Iter       Train Loss   Remaining Time 
         1     8668381.1560            1.01s
         2     7527263.4187            0.95s
         3     6578597.1849            0.90s
         4     5770589.0947            0.86s
         5     5087152.9130            0.83s
         6     4510100.2753            0.80s
         7     4031858.3436            0.76s
         8     3634256.0016            0.73s
         9     3296767.1062            0.69s
        10     3007253.3094            0.66s
        20     1627021.2128            0.32s
        30     1220796.5502            0.00s
Model training took:  0.96  seconds

MODEL ACCURACY METRICS WITH TEST DATASET: 
	 Root mean squared error: 1231 
	 Mean absolute error: 910 
	 Coefficient of determination: 0.8383 



## 6. Random Forest Regressor

* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
* https://scikit-learn.org/stable/modules/ensemble.html#forest

Cannot extrapolate beyond training dataset; mostly used for classification

In [7]:
random_forest = RandomForestRegressor(n_estimators=30,verbose=1)
random_forest_name = "Random Forest Regressor"

random_forest = train_model(x_train, y_train,random_forest)
random_forest_predictions = random_forest.predict(x_test)
metrics_collection = print_and_store_error_metrics(y_test,random_forest_predictions, random_forest_name, metrics_collection)

RandomForestRegressor(n_estimators=30, verbose=1)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model training took:  2.28  seconds

MODEL ACCURACY METRICS WITH TEST DATASET: 
	 Root mean squared error: 1181 
	 Mean absolute error: 861 
	 Coefficient of determination: 0.8511 



[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


In [ ]:

#let's take a look at feature importances
feature_importances = pd.DataFrame(random_forest.feature_importances_, index =random_forest.columns,  columns=['importance']).sort_values('importance', ascending=False)
feature_importances.head()

## 7. Extra Trees Regressor

* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html

In [8]:
extra_trees = ExtraTreesRegressor(n_estimators=30,verbose=1)
extra_trees_name = "Extra Trees Regressor"

extra_trees = train_model(x_train, y_train,extra_trees)
extra_trees_predictions = extra_trees.predict(x_test)
metrics_collection = print_and_store_error_metrics(y_test,extra_trees_predictions, extra_trees_name, metrics_collection)

ExtraTreesRegressor(n_estimators=30, verbose=1)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model training took:  1.02  seconds

MODEL ACCURACY METRICS WITH TEST DATASET: 
	 Root mean squared error: 1143 
	 Mean absolute error: 832 
	 Coefficient of determination: 0.8605 



[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


## 8. Bagging Regressor

* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html
* https://scikit-learn.org/stable/modules/ensemble.html#bagging

In [9]:
bagging = BaggingRegressor(n_estimators=30,verbose=1)
bagging_name = "Bagging Regressor"

baggings = train_model(x_train, y_train,bagging)
bagging_predictions = bagging.predict(x_test)
metrics_collection = print_and_store_error_metrics(y_test,bagging_predictions, bagging_name, metrics_collection)

BaggingRegressor(n_estimators=30, verbose=1)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model training took:  2.29  seconds

MODEL ACCURACY METRICS WITH TEST DATASET: 
	 Root mean squared error: 1204 
	 Mean absolute error: 872 
	 Coefficient of determination: 0.8453 



[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 8. AdaBoost Regressor

* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html
* https://scikit-learn.org/stable/modules/ensemble.html#adaboost

In [10]:
ada_boost = AdaBoostRegressor(n_estimators=30)
ada_boost_name = "AdaBoost Regressor"

ada_boost = train_model(x_train, y_train,ada_boost)
ada_boost_predictions = ada_boost.predict(x_test)
metrics_collection = print_and_store_error_metrics(y_test,ada_boost_predictions, ada_boost_name, metrics_collection)

AdaBoostRegressor(n_estimators=30)
Model training took:  0.72  seconds

MODEL ACCURACY METRICS WITH TEST DATASET: 
	 Root mean squared error: 1394 
	 Mean absolute error: 1099 
	 Coefficient of determination: 0.7927 



## 9. Model comparison

In [ ]:
print(metrics_collection.sort_values(by=['RMSE'], ascending=False))

## 10. Task

Study the scikit-learn documentation of one of the above used models and experiment with different hyperparameter values. Can you improve on the accuracy or make the training faster?
Report the best results and mark down parameters used, so that others can reproduce the results.